In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
!pip install -U -r yolov5/requirements.txt  # install dependencies
%cd /content/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16491, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 16491 (delta 27), reused 35 (delta 9), pack-reused 16408
Receiving objects: 100% (16491/16491), 15.11 MiB | 29.75 MiB/s, done.
Resolving deltas: 100% (11292/11292), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 795.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 56.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 54.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 6

# Dataset Split

Preparing the data structure for training, validation and testing

In [2]:
import pandas as pd
import shutil
import os

# Define the base directory paths
base_dir = '/kaggle/input/fracatlas/FracAtlas'
images_dir = os.path.join(base_dir, 'images', 'Fractured')  # Path to the original images
splits_dir = os.path.join(base_dir, 'Utilities', 'Fracture Split')  # Path to the CSV files

# Define the target directories for images
target_images_dir = os.path.join('/kaggle/working/')
train_dir = os.path.join(target_images_dir, 'train', 'images')
val_dir = os.path.join(target_images_dir, 'val', 'images')
test_dir = os.path.join(target_images_dir, 'test', 'images')

# Create target directories if they don't exist
for directory in [train_dir, val_dir, test_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to copy images based on CSV file
def copy_images(csv_file, target_dir):
    df = pd.read_csv(csv_file)
    for index, row in df.iterrows():
        image_file = row['image_id']  # Assuming 'image_name' column contains the image filenames
        source_path = os.path.join(images_dir, image_file)
        target_path = os.path.join(target_dir, image_file)
        shutil.copy(source_path, target_path)

# Copy images for each split
copy_images(os.path.join(splits_dir, 'train.csv'), train_dir)
copy_images(os.path.join(splits_dir, 'valid.csv'), val_dir)
copy_images(os.path.join(splits_dir, 'test.csv'), test_dir)

print("Images have been copied to the respective folders.")


Images have been copied to the respective folders.


Creating YOLO labels from COCO Annotations

In [3]:
import json
import os
import pandas as pd

# Paths to the COCO annotation file, CSV files for splits, and the images directory
coco_annotation_path = '/kaggle/input/fracatlas/FracAtlas/Annotations/COCO JSON/COCO_fracture_masks.json'
train_csv_path = '/kaggle/input/fracatlas/FracAtlas/Utilities/Fracture Split/train.csv'
val_csv_path = '/kaggle/input/fracatlas/FracAtlas/Utilities/Fracture Split/valid.csv'
test_csv_path = '/kaggle/input/fracatlas/FracAtlas/Utilities/Fracture Split/test.csv'
images_base_path = '/kaggle/input/fracatlas/FracAtlas/images/Fractured'  # Update this path

# Base directory where the YOLO labels should be saved
base_save_path = '/kaggle/working/'

def coco_to_yolo(box, img_w, img_h):
    x, y, w, h = box
    return [((x + w / 2) / img_w), ((y + h / 2) / img_h), (w / img_w), (h / img_h)]

def load_splits():
    splits = {'train': pd.read_csv(train_csv_path),
              'val': pd.read_csv(val_csv_path),
              'test': pd.read_csv(test_csv_path)}
    image_to_split = {}
    for split_name, split_df in splits.items():
        for img_id in split_df['image_id']:  # Adjust if the column name differs
            image_to_split[img_id] = split_name
    return image_to_split

image_to_split = load_splits()
# print('Image to Split',image_to_split)

with open(coco_annotation_path, 'r') as f:
    coco_data = json.load(f)

for img in coco_data['images']:
    img_id = img['id']  # Use image ID to match with splits
#     print('Img_id',img_id)
    img_name = os.path.splitext(img['file_name'])[0]
    split = image_to_split[img_name+'.jpg']
    img_path = os.path.join(images_base_path, img['file_name'])  # Full path to the image file
#     print('img_path',img_path)

    # Proceed if the image exists
    img_w, img_h = img['width'], img['height']
    annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == img_id]
#     print('Annotation',annotations)
    yolo_labels = []
    
    for ann in annotations:
        category_id = ann['category_id']  # Adjust based on your category mapping
        box = coco_to_yolo(ann['bbox'], img_w, img_h)
        yolo_labels.append([category_id, *box])
    
    labels_save_path = os.path.join(base_save_path, split, 'labels', f'{img_name}.txt')
    os.makedirs(os.path.dirname(labels_save_path), exist_ok=True)
#     print('yolo_labels', yolo_labels)
    with open(labels_save_path, 'w') as label_file:
        for label in yolo_labels:
            label_file.write(' '.join(map(str, label)) + '\n')

print("Conversion complete. Labels saved in YOLO format for existing images.")


Conversion complete. Labels saved in YOLO format for existing images.


## Training

In [5]:
yaml_content = """
path: /kaggle/working/
train: /kaggle/working/train
val: /kaggle/working/val
nc: 1  # Number of classes, adjust as per your dataset
names: ['Fractured']  # Adjust class names according to your dataset
"""

file_path = '/kaggle/working/fracatlas_dataset.yml'  # Specify your desired file path

with open(file_path, 'w') as file:
    file.write(yaml_content)


In [6]:
import torch
from IPython.display import Image  # for displaying images
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 2.2.1+cu121 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [7]:
cd "/kaggle/working/yolov5/"

/kaggle/working/yolov5


Training the YOLO model

In [8]:
!python train.py --img 640 --batch 16 --epochs 25 --data /kaggle/working/fracatlas_dataset.yml --weights yolov5l6.pt --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-02-27 18:08:02.569056: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 18:08:02.569178: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 18:08:02.883741: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train:

Detection of Fractures in Test Images

In [9]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --source /kaggle/working/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=/kaggle/working/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-287-g574331f9 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 346 layers, 76118664 parameters, 0 gradients, 109.9 GFLOPs
image 1/61 /kaggle/working/test/images/IMG0003297.jpg: 640x576 (no detections), 89.5ms
image 2/61 /kaggle/working/test/images/IMG0003298.jpg: 640x576 1 Fractured, 46.7ms
image 3/61 /kaggle/working/test/images/IMG0003301.jpg: 640x576 (no detections), 46.9ms
image 4/61 /kaggle/working/test/images/IMG0003308.jpg: 640x576

In [10]:
yaml_content = """
path: /kaggle/working/test  # Base path for the dataset
train: /kaggle/working/train
val: /kaggle/working/val
test: /kaggle/working/test  # Relative path from `path` to the test images
nc: 1  # Number of classes
names: ['Fractured']  # Names of the classes
"""

file_path = '/kaggle/working/fracatlas_test_dataset.yml'  # Specify your desired file path

with open(file_path, 'w') as file:
    file.write(yaml_content)


Evaluating the model on Test Image to obtain the metrics

In [11]:
!python val.py --data /kaggle/working/fracatlas_test_dataset.yml --weights runs/train/exp/weights/best.pt --img 640 --task test --conf-thres 0.25 --iou-thres 0.5

val: data=/kaggle/working/fracatlas_test_dataset.yml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.25, iou_thres=0.5, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.25 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-287-g574331f9 Python-3.10.13 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 346 layers, 76118664 parameters, 0 gradients, 109.9 GFLOPs
test: Scanning /kaggle/working/test/labels... 61 images, 0 backgrounds, 0 corrup
test: New cache created: /kaggle/working/test/labels.cache
                 Class     Images  Instances          P          R      mAP50   
                   all         61         67      0.663      0.433      0.521      0.258
Speed: 0.3ms pre-process, 27.9ms inference, 9.3ms NMS p